<a href="https://colab.research.google.com/github/homosapienssapiens/feature-engineering/blob/main/Final%20project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Proyecto final
Miguel Angel Solis Orozco  
Ingeniería de características  
Instituto de Estudios Superiores de Occidente  
Prof: Dr. Carlos Augusto Arellano Muro

---



1. Separe las variables x1 y x2 en columnas diferentes.

In [ ]:
import pandas as pd

In [ ]:
route = "/content/drive/MyDrive/Maestría en ciencia de datos/2do semestre/Ingeniería de características/Trabajo final/"
df = pd.read_csv(route + "data-1.csv")

In [ ]:
df

,nombre,Fecha,data
0,x1,2006-01-01,6.05
1,x1,2006-01-02,4.72
2,x1,2006-01-03,4.46
3,x1,2006-01-04,34.74
4,x1,2006-01-05,1.91
...,...,...,...
6566,x2,27/12/2014,-13.30
6567,x2,28/12/2014,16.20
6568,x2,29/12/2014,5.10
6569,x2,30/12/2014,4.90


Analizando el DataFrame podemos ver que las fechas no se encuentran en el formato fecha correcto. A continuación pondremos el formato a date time.

In [ ]:
df['Fecha'] = pd.to_datetime(df['Fecha'])
df

,nombre,Fecha,data
0,x1,2006-01-01,6.05
1,x1,2006-01-02,4.72
2,x1,2006-01-03,4.46
3,x1,2006-01-04,34.74
4,x1,2006-01-05,1.91
...,...,...,...
6566,x2,2014-12-27,-13.30
6567,x2,2014-12-28,16.20
6568,x2,2014-12-29,5.10
6569,x2,2014-12-30,4.90


In [ ]:
dfsplit = pd.DataFrame(columns = ['Date', 'x1', 'x2'])
x1 = float('NaN')
x2 = float('NaN')
mydates = pd.date_range(min(df['Fecha']), max(df['Fecha'])).tolist()
for i in mydates:
  for j in df.index:
    if df.loc[j, 'Fecha'] == i:
      fecha = df.loc[j, 'Fecha']
      if df.loc[j, 'nombre'] == 'x1': 
        x1 = df.loc[j, 'data']
      else:
        x2 = df.loc[j, 'data']
  row = {'Date': fecha,
    'x1': x1,
    'x2': x2}
  dfsplit = dfsplit.append(row, ignore_index = True)
  x1 = float('NaN')
  x2 = float('NaN')
dfsplit

,Date,x1,x2
0,2006-01-01,6.05,9.22
1,2006-01-02,4.72,1.14
2,2006-01-03,4.46,-13.46
3,2006-01-04,34.74,3.33
4,2006-01-05,1.91,-0.14
...,...,...,...
3282,2014-12-27,13.16,-13.30
3283,2014-12-28,8.50,16.20
3284,2014-12-29,11.03,5.10
3285,2014-12-30,21.33,4.90


2. Elija y justifique un método de imputación de datos para los datos perdidos y los datos menores o iguales que cero.